In [1]:
import time
import psycopg2
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col, date_format, broadcast
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, TimestampType, MapType
from elasticsearch import Elasticsearch, exceptions

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
2,application_1724580650220_0003,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
# Elasticsearch configuration
es_host = "10.0.3.216"
es_port = 9200  # Ensure this is an integer
es_scheme = "http"


# Create Elasticsearch client
es = Elasticsearch([{'host': es_host, 'port': es_port, 'scheme': es_scheme}])

SEVs_mappings = {
    "properties": {
          "SEV_ID": {
            "type": "keyword"
          },
          "VIN": {
            "type": "keyword"
          },
          "Timestamp": {
            "type": "text"  # Changed to text
          },
          "Name": {
            "type": "text"
          },
          "Type": {
            "type": "text"
          },
          "Severity": {
            "type": "keyword"
          },
          "NetworkType": {
            "type": "keyword"
          },
          "NetworkID": {
            "type": "keyword"
          },
          "SEV_Msg": {
            "type": "text"
          },
          "Origin": {
            "type": "keyword"
          },
          "IoC": {
            "type": "object"
          }
    }
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
if es.indices.exists(index="joined_raw_index"):
    es.indices.delete(index="joined_raw_index")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
# Create SparkSession
spark = SparkSession.builder \
    .appName("KafkaToElasticsearch") \
    .config("spark.sql.streaming.checkpointLocation", "s3://aws-emr-studio-381492251123-eu-central-1/stream_checkpoint/checkpoint/") \
    .getOrCreate()

# JDBC properties
jdbc_url = "jdbc:postgresql://10.0.3.216:5432/test_db"
jdbc_properties = {
    "user": "postgres",
    "password": "postgres",
    "driver": "org.postgresql.Driver"
}

# Read static data from PostgreSQL
car_model_df = spark.read.jdbc(url=jdbc_url, table="cars", properties=jdbc_properties)

# Rename column in the PostgreSQL DataFrame to match Kafka stream DataFrame
car_model_df = car_model_df.withColumnRenamed("VIN Number", "VIN")


car_model_df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----------------+
|           Car Model|              VIN|
+--------------------+-----------------+
|Mercedes-Benz C-C...|5ZC7FP1CELU542056|
|       Nissan Altima|THM35QS7GNW787602|
|          Mazda CX-5|FCJBK973DHT960860|
|Mercedes-Benz C-C...|FBKNVQPO3P8916909|
|      Subaru Outback|3BV3J0SX765828596|
|     Hyundai Elantra|KXAAA66Z49K588982|
|        Toyota Camry|LKNZM5U782A364319|
|      Subaru Outback|DLDMEC1HJTJ357517|
|        Honda Accord|1TW2ZQDESAS678424|
|        BMW 3 Series|DFUJL033LLU546604|
|          Mazda CX-5|KBTBNV01AYK113631|
|        BMW 3 Series|UKAR97F5FTE400236|
|        Ford Mustang|PXH6QA41EYL676540|
|       Nissan Altima|XPH52B0UNEB437989|
|        Toyota Camry|6VBH7XD5361950961|
|        Honda Accord|BCK7BFM186V025443|
|        Toyota Camry|NGCDQU8TCBH013582|
|     Hyundai Elantra|MYUMYC7WGGW452143|
|       Nissan Altima|2DLXV2IWY9C152840|
|    Chevrolet Malibu|WLT7C5ZYGJ2677014|
+--------------------+-----------------+
only showing top

In [5]:
def create_index_if_not_exists(es_client, index_name, mappings):
    try:
        if not es_client.indices.exists(index=index_name):
            print(f"Index '{index_name}' does not exist. Creating index...")
            es_client.indices.create(
                index=index_name,
                body={
                    "mappings": mappings  # Use the provided mappings parameter
                }
            )
            print(f"Index '{index_name}' created successfully.")
        else:
            print(f"Index '{index_name}' already exists.")
    except exceptions.RequestError as e:
        print(f"RequestError: {e.info}")
    except exceptions.ConnectionError as e:
        print(f"ConnectionError: {e}")
    except Exception as e:
        print(f"Error creating index: {e}")

# Create the index in Elasticsearch
create_index_if_not_exists(es, "sevs_index", SEVs_mappings)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Index 'sevs_index' already exists.

In [6]:
# Define the schema
# Define the schema for the IoC field as a MapType to accept any JSON
ioc_schema = MapType(StringType(), StringType())

# Define the full schema for the data
schema = StructType([
    StructField("SEV_ID", StringType(), True),
    StructField("VIN", StringType(), True),
    StructField("Timestamp", TimestampType(), True),
    StructField("Name", StringType(), True),
    StructField("Type", StringType(), True),
    StructField("Severity", StringType(), True),
    StructField("NetworkType", StringType(), True),
    StructField("NetworkID", StringType(), True),
    StructField("SEV_Msg", StringType(), True),
    StructField("Origin", StringType(), True),
    StructField("IoC", ioc_schema, True)
])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
from pyspark.sql.functions import from_json, col, date_format
from pyspark.sql.types import StructType, StructField, StringType, TimestampType, MapType
from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.builder \
    .appName("SEVsToElasticsearch") \
    .getOrCreate()

# Kafka parameters
kafka_bootstrap_servers = "b-1.kafkaclustersevs.pgxp3x.c3.kafka.eu-central-1.amazonaws.com:9092,b-2.kafkaclustersevs.pgxp3x.c3.kafka.eu-central-1.amazonaws.com:9092"
kafka_topic = "topic2"

# Create a DataFrame representing the stream of input lines from Kafka
kafka_df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", kafka_bootstrap_servers) \
    .option("subscribe", kafka_topic) \
    .option("startingOffsets", "earliest") \
    .load()

# Convert the value column from Kafka to a string
kafka_df = kafka_df.selectExpr("CAST(value AS STRING)")

# Parse the JSON data using the schema
kafka_df = kafka_df.select(from_json(col("value"), schema).alias("data"))

# Flatten the DataFrame
kafka_df = kafka_df.select(
    col("data.SEV_ID"),
    col("data.VIN"),
    date_format(col("data.Timestamp"), "yyyy-MM-dd'T'HH:mm:ss.SSSSSS").alias("Timestamp"),
    col("data.Name"),
    col("data.Type"),
    col("data.Severity"),
    col("data.NetworkType"),
    col("data.NetworkID"),
    col("data.SEV_Msg"),
    col("data.Origin"),
    col("data.IoC")
)

# Broadcast the static DataFrame
broadcast_car_model_df = broadcast(car_model_df)

# Perform the join with the Kafka stream DataFrame
joined_df = kafka_df.join(broadcast_car_model_df, kafka_df.VIN == broadcast_car_model_df.VIN, "left_outer") \
    .select(
        kafka_df.SEV_ID,
        kafka_df.VIN,
        kafka_df.Timestamp,
        kafka_df.Name,
        kafka_df.Type,
        kafka_df.Severity,
        kafka_df.NetworkType,
        kafka_df.NetworkID,
        kafka_df.SEV_Msg,
        kafka_df.Origin,
        kafka_df.IoC,
        broadcast_car_model_df["Car Model"].alias("Car Model")
    )

# Elasticsearch configuration for Spark
es_write_conf = {
    "es.nodes": es_host,
    "es.port": str(es_port),  # Convert port to string for Spark configuration
    "es.index.auto.create": "true"  # Ensure that Spark does not attempt to auto-create the index
}

# Write the joined data to Elasticsearch
query = joined_df.writeStream \
    .format("org.elasticsearch.spark.sql") \
    .options(**es_write_conf) \
    .option("es.resource", "sevs_index") \
    .outputMode("append") \
    .start()

# Await termination of the stream
query.awaitTermination()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkStatementCancellationFailedException: Interrupted by user but Livy failed to cancel the Spark statement. The Livy session might have become unusable.